In [10]:
from utils.data_cleaning import *
from utils.data_preprocess import *
from utils.evaluate import evaluate_model
from utils.cf_utils import *
from utils.models import *
from utils.big_data_helper import *
import polars as pl

from utils.maps_creater import *


import faiss
from tqdm import tqdm
from sklearn.pipeline import Pipeline


# Data

### DataLoding

In [30]:
data = pl.scan_parquet("data/multi_event.parquet")

In [3]:
import polars as pl

estimate_parquet_ram_usage("data/multi_event.parquet")


📁 Файл: data/multi_event.parquet
📊 Сэмпл строк: 10,000
🔹 Размер sample: 0.19 MB
🔹 Всего строк в файле: 47,790,449

📐 Примерная оценка объёма данных в памяти: 0.90 GB


### Train/test split

Для начала разделим данные на обучающую и тестовую выборки. Деление выполняется по временному признаку, поскольку мы работаем с последовательностями действий пользователей.

Чтобы избежать data leakage — ситуации, когда признаки, рассчитанные на исторических данных, могут «заглядывать» в будущее — вводится временной зазор (gap) между обучающей и тестовой частями.

В оригинальной статье https://www.arxiv.org/pdf/2505.22238  используется соотношение 300 дней обучения, 30 минут gap, 1 день теста.

В моих экспериментах наилучшие результаты получились при конфигурации: 80 дней обучения, 30 минут gap, 20 дней теста.

Чтобы приблизиться к условиям статьи, я планирую снижать размер тестовой выборки, сохраняя возможность варьировать объём обучающей части как гиперпараметр модели. Такой подход позволяет гибко контролировать баланс между длиной обучающего периода и реалистичностью оценки качества.

In [31]:
train_lf, test_lf = train_test_split(data, 1, gap_size = 30)
train_lf.sink_parquet("data/work/train_df.parquet")
test_lf.sink_parquet("data/work/test_df.parquet")

In [5]:
estimate_parquet_ram_usage("data/work/train_df.parquet")
estimate_parquet_ram_usage("data/work/test_df.parquet")


📁 Файл: data/work/train_df.parquet
📊 Сэмпл строк: 10,000
🔹 Размер sample: 0.19 MB
🔹 Всего строк в файле: 47,550,037

📐 Примерная оценка объёма данных в памяти: 0.90 GB

📁 Файл: data/work/test_df.parquet
📊 Сэмпл строк: 10,000
🔹 Размер sample: 0.19 MB
🔹 Всего строк в файле: 238,398

📐 Примерная оценка объёма данных в памяти: 0.00 GB


### Очиска и кодирование данных 
Применим необходимую предобработку для очистки и подготовки данных. Подробные рассуждения о шагах и мотивации предобработки приведены в блокноте multi_event_preprocessing.
Все функции вынесены в отдельный модуль, что делает код более читаемым, модульным и удобным для повторного использования.

Кодируем uid и item_id в числовые индексы, чтобы модели и алгоритмы могли с ними работать.

Строковые ID нельзя напрямую подать в матрицу взаимодействий или Embedding-слой, поэтому мы переводим их в диапазон 0..N.
Это нужно для эффективности (память, скорость), совместимости с моделями (Matrix Factorization, Embeddings) и стабильного маппинга между train и test.

In [96]:
train_lf = pl.scan_parquet("data/work/train_df.parquet")
test_lf = pl.scan_parquet("data/work/test_df.parquet")

In [2]:
# создаём словари индексов
user_map, item_map = build_id_maps(train_lf)

# мапим train/test
train_encoded_lf = map_with_id_maps(train_lf, user_map, item_map)
test_encoded_lf  = map_with_id_maps(test_lf,  user_map, item_map)

train_encoded_lf.sink_parquet("data/work/train_encoded_lf.parquet")
test_encoded_lf.sink_parquet("data/work/test_encoded_lf.parquet")

NameError: name 'train_lf' is not defined

#### Сохраняем мар-ы для повторного кодирования

In [45]:
import json

with open("data/item_map.json", "w", encoding="utf-8") as f:
    json.dump(item_map, f, ensure_ascii=False, indent=2)

with open("data/user_map.json", "w", encoding="utf-8") as f:
    json.dump(item_map, f, ensure_ascii=False, indent=2)

In [46]:
apply_function_by_batch("data/work/train_encoded_lf.parquet", 
                         "data/work/train_df_listen.parquet", 
                         get_listen_data, 
                         "timestamp", 
                         batch_size = 10_000_000)

apply_function_by_batch("data/work/test_encoded_lf.parquet", 
                         "data/work/test_df_listen.parquet", 
                         get_listen_data , 
                         "timestamp", 
                         batch_size = 10_000_000)

apply_function_by_batch("data/work/train_encoded_lf.parquet", 
                         "data/work/train_df_likes.parquet", 
                         get_not_listen_data, 
                         "timestamp", 
                         batch_size = 10_000_000)

apply_function_by_batch("data/work/test_encoded_lf.parquet", 
                         "data/work/test_df_likes.parquet", 
                         get_not_listen_data , 
                         "timestamp", 
                         batch_size = 10_000_000)

In [47]:
# Из трейна remove_duplicates_by_timestamps
apply_function_by_batch("data/work/train_df_listen.parquet", 
                         "data/work/train_df_listen_1.parquet", 
                         remove_duplicates_by_timestamps , 
                         "timestamp", 
                         batch_size = 10_000_000)

# Из трейна filter_rare_items
apply_function_by_batch("data/work/train_df_listen_1.parquet", 
                         "data/work/train_df_listen_2.parquet", 
                         filter_rare_items , 
                         "item_id", 
                         batch_size = 10_000_000)

# Из трейна filter_rare_users
apply_function_by_batch("data/work/train_df_listen_2.parquet", 
                         "data/work/train_df_listen_3.parquet", 
                         filter_rare_users , 
                         "uid", 
                         batch_size = 10_000_000)

# Из трейна cut_track_len
apply_function_by_batch("data/work/train_df_listen_3.parquet", 
                         "data/work/train_df_listen_4.parquet", 
                         cut_track_len , 
                         "timestamp", 
                         batch_size = 10_000_000)

In [48]:
# Из трейна convert_reaction
apply_function_by_batch("data/work/train_df_likes.parquet", 
                         "data/work/train_df_likes_1.parquet", 
                         convert_reaction , 
                         "uid", 
                         batch_size = 10_000_000)

# Из трейна rename_events
apply_function_by_batch("data/work/train_df_likes_1.parquet", 
                         "data/work/train_df_likes_2.parquet", 
                         rename_events , 
                         "timestamp", 
                         batch_size = 10_000_000)

In [49]:
# Из теста convert_reaction
apply_function_by_batch("data/work/test_df_likes.parquet", 
                         "data/work/test_df_likes_1.parquet", 
                         convert_reaction , 
                         "uid", 
                         batch_size = 10_000_000)

# Из теста rename_events
apply_function_by_batch("data/work/test_df_likes_1.parquet", 
                         "data/work/test_df_likes_2.parquet", 
                         rename_events , 
                         "timestamp", 
                         batch_size = 10_000_000)

In [50]:
concat_files("data/work/test_df_listen.parquet", "data/work/test_df_likes_2.parquet", "data/work/test_df_preprocessed.parquet")

concat_files("data/work/train_df_listen_4.parquet", "data/work/train_df_likes_2.parquet", "data/work/train_df_preprocessed.parquet")


In [2]:
estimate_parquet_ram_usage("data/work/train_df_preprocessed.parquet")
estimate_parquet_ram_usage("data/work/test_df_preprocessed.parquet")


📁 Файл: data/work/train_df_preprocessed.parquet
📊 Сэмпл строк: 10,000
🔹 Размер sample: 0.27 MB
🔹 Всего строк в файле: 31,907,792

📐 Примерная оценка объёма данных в памяти: 0.83 GB

📁 Файл: data/work/test_df_preprocessed.parquet
📊 Сэмпл строк: 10,000
🔹 Размер sample: 0.27 MB
🔹 Всего строк в файле: 237,786

📐 Примерная оценка объёма данных в памяти: 0.01 GB


#### Map для "уже послушанных песен пользователя"

Так как цель предсказывать что-то новое, что нужно фильтровать песни которые пользователь уже видел из рекомендаций, а так же не учитывать их при тестировании. При тестировании список прослушанных песен пользователя должен фильтровался от уже прослушанных, так как система не предсказывает такие песни и нет смысла учитывать их при тестировании. Пользователи которые не слушают новые песни не включаются в оценки (модель не может угадывать то, чего не было). Для учета таких случаев будет введена отдельная метрика, показывающая вовлеченность пользователей в рекомендательную систему.

In [2]:
# Береме данные до препроцессинка (но закодированные)
history =  pl.scan_parquet("data/work/train_encoded_lf.parquet")

test_df = pl.scan_parquet("data/work/test_df_preprocessed.parquet")
test_users_items_df = remove_listened_data(test_df)

test_users_items_df.sink_parquet("data/work/test_df_preprocessed_for_eval.parquet")


In [11]:
train_df = pl.scan_parquet("data/work/train_df_preprocessed.parquet")
test_df  = pl.scan_parquet("data/work/test_df_preprocessed_for_eval.parquet") 
history =  pl.scan_parquet("data/work/train_encoded_lf.parquet")
users_history = build_users_history(history, last_days=30)

# Models

## Baseline

#### Глобальные тренды

Самый простой базовый подход — взять топ-10 самых популярных треков. При этом возможны два варианта отбора:

* по количеству лайков,
* по количеству прослушиваний.

Реализованы обе версии, однако в качестве базовой модели (baseline) выбран именно топ-10 самых залайканных треков среди всех пользователей. Лайки считаются более надёжным сигналом предпочтений, чем прослушивания. (Для чистоты эксперимента был протестирован и вариант с прослушиваниями — его точность оказалась ниже.)



Поскольку тренды и популярные песни сменяются очень быстро, временное окно, в котором мы определяем тренд, также должно быть коротким. (*PS нужно оптимизировать выбор автоматически*)

In [3]:
model = MostPop_by_likes()
model.fit(train_df)
evaluate_model(model, users_history, test_df)

100%|██████████████████████████████████████████████████████████████████████████████| 4827/4827 [00:00<00:00, 51322.24it/s]

Mean Recall@10: 0.01540002555183077
Mean NDCG@10: 0.026082193739347365


In [17]:
model = MostPop_by_listen()
model.fit(train_df)

evaluate_model(model, users_history, test_df)

100%|██████████████████████████████████████████████████████████████████████████████| 4827/4827 [00:00<00:00, 50589.47it/s]

Mean Recall@10: 0.01248428881344854
Mean NDCG@10: 0.02367576645027094


#### Новинки последних дней

Похоже на модель популярных песен, но берет только новые песни. В топ прослушаных могут попадать к примеру песня Шуфутинского 3го сентября 

In [5]:
model = NewItemsLastNDays()
model.fit(train_df)

evaluate_model(model, users_history, test_df)

100%|██████████████████████████████████████████████████████████████████████████████| 4827/4827 [00:00<00:00, 51998.04it/s]

Mean Recall@10: 0.006657907090237797
Mean NDCG@10: 0.015225396394740779


#### Последнее прослушанное

Скорее всего пользователь в ближайщем времени послушает то, что слушал некоторое время назад. Такое предсказние дает большой процент точности, но не совсем то что нужно для рекомендательной системы, у которой цель посоветовать что-то новое и не из последний песен плейлиста. Такие песни можно будет исключать в будущем 

In [6]:
test_df  = pl.scan_parquet("data/work/test_df_preprocessed_for_eval.parquet") 
train_df = pl.scan_parquet("data/work/train_df_preprocessed.parquet")

model = LastListenRecommender()


hour = 24
decay = 0.9
tau = 0.0 if hour == 0 else decay ** (1 / 24 / 60 / 60 / (hour / 24))

df_tau = add_exponential_decay(train_df, tau)

model.fit(df_tau)

evaluate_model(model, users_history, test_df, 50) # 50 потому что в планах фильтровать последние 50 прослушанных 

100%|████████████████████████████████████████████████████████████████████████████████| 4827/4827 [00:18<00:00, 267.68it/s]

Mean Recall@50: 0.23084021504394486
Mean NDCG@50: 0.20862530927606193


## Collaborative filtering

### ALS

AlternatingLeastSquares (ALS) — это модель матричной факторизации для неявных откликов (implicit feedback).
Она раскладывает матрицу взаимодействий пользователей и объектов на две матрицы эмбеддингов — пользователей и айтемов — так, чтобы их скалярное произведение предсказывало интерес.
Обучается поочерёдным решением задач наименьших квадратов: сначала фиксирует айтемы и обновляет пользователей, потом наоборот.


В одной итерации ALS поочерёдно пересчитывает вектора пользователей и айтемов.
Сначала, при фиксированных айтемах, решаются задачи наименьших квадратов для всех пользователей;
потом, при фиксированных пользователях — для всех айтемов.
Это повторяется несколько итераций, пока модель не сойдётся.

Для обучения ALS требуется подготовить данные в формате матрицы взаимодействий, где каждая ячейка отражает степень интереса пользователя к объекту. Поскольку в нашем случае используются implicit-сигналы, нам необходимо преобразовать исходные события (прослушивание, лайк, дизлайк) в числовые веса.

На этапе подготовки данных мы подсчитываем, сколько раз пользователь прослушал каждый трек, а также приводим реакции к бинарному виду:
в результирующем датафрейме поле like получает значение 1,
аналогично dislike — значение 1.

В отдельном модуле реализована функция объединения данных, которая формирует итоговый датафрейм с колонками:

* признак лайка (like = 1)
* признак дизлайка (dislike = 1)
* количество прослушиваний каждой пары пользователь–трек

In [8]:

train_df_conf = merge_data_by_count(train_df)
train_df_conf = calculate_conf(train_df_conf)

(
    train_df_conf
    .collect()  # выполняем план, получаем обычный DataFrame
    .write_parquet("data/work/train_df_conf.parquet")
)

In [9]:
estimate_parquet_ram_usage("data/work/train_df_conf.parquet")


📁 Файл: data/work/train_df_conf.parquet
📊 Сэмпл строк: 10,000
🔹 Размер sample: 0.36 MB
🔹 Всего строк в файле: 7,298,676

📐 Примерная оценка объёма данных в памяти: 0.26 GB


Для ALS необходима разреженная матрица вида:

(user_id, item_id) → weight


где weight — это итоговая сила взаимодействия (прослушивания + реакции).
В модуле реализована функция, которая на основе сформированного датафрейма строит такую матрицу через coo_matrix и возвращает её в подходящем для модели формате.

In [204]:
train_df_conf = pl.scan_parquet("data/work/train_df_conf.parquet")

# train_df_conf = train_df_conf.with_columns(
#     (pl.col("conf") != 0).cast(pl.Int8).alias("conf")
# )

model = ALS()
model.fit(train_df_conf)



/opt/venv/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 16 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|███████████████████████████████████████████████████████████████████████| 15/15 [00:27<00:00,  1.84s/it, loss=0.00264]


In [205]:
evaluate_model(model, users_history, test_df)

100%|█████████████████████████████████████████████████████████████████████████████████| 4827/4827 [01:53<00:00, 42.65it/s]


Mean Recall@10: 0.017005667891855
Mean NDCG@10: 0.04118527317112505


Метод ALS работает только с пользователями и объектами, которые присутствовали в обучающей выборке. Если в тесте встречается пользователь, которого модель ранее не видела, для него невозможно построить профиль в факторизационном пространстве.

В такой ситуации ALS не способен выдать рекомендации — в классе ALS такая логика приводит к возврату пустого списка.

Поэтому при использовании ALS важно учитывать, что:

* рекомендации гарантированно работают только для пользователей из train,
* все новые или ранее невстречавшиеся пользователи будут давать «пустой» результат.

Это естественное ограничение моделей матричной факторизации, основанных на implicit-feedback данных.

### BM25Rec

In [75]:
model = BM25Rec()
model.fit(train_df_conf)
evaluate_model(model, users_history, test_df)


### ItemKNN

In [209]:
model = ItemKNN()
model.fit(train_df)
evaluate_model(model, users_history, test_df, k=10)

100%|█████████████████████████████████████████████████████████████████████████████████| 4827/4827 [01:46<00:00, 45.44it/s]

Mean Recall@10: 0.0431855077752173
Mean NDCG@10: 0.08053783579665744


 В памяти все хранить очень тяжело, но по математическим расчетам это тоже самое, поэтому используем
 Конечно тоже не быстро но лучше чем было

### BPR

In [210]:
train_df_conf = pl.scan_parquet("data/work/train_df_conf.parquet")

train_df_conf = train_df_conf.with_columns(
    (pl.col("conf") != 0).cast(pl.Int8).alias("conf")
)

model = BPR()
model.fit(train_df_conf)

evaluate_model(model, users_history, test_df)

100%|█████████████████████████████████████████████████████████████████████████████████| 4827/4827 [01:57<00:00, 41.08it/s]

Mean Recall@10: 0.014311525458997233
Mean NDCG@10: 0.030514844986110252


## Graph-based recommendation

In [26]:
import json 
train_df = pl.scan_parquet("data/work/train_df_preprocessed.parquet")


df_merged = merge_data_by_count(train_df)


with open("data/item_map.json", "r", encoding="utf-8") as f:
    item_map = json.load(f)

item_map = {int(k): v for k, v in item_map.items()}

items_meta = (
    pl.scan_parquet("data/items_meta.parquet")
    .with_columns(
        pl.col("item_id").replace(item_map)
    )
    .unique(subset=["item_id"])
    .drop_nulls()
)

lf_user_track = df_merged.join(
    items_meta.select(["item_id", "artist_id", "album_id"]),
    on="item_id",
    how="left",
)

In [60]:
model = RandomWalkWithRestart()
model.fit(lf_user_track)
# test_users_items_df.filter(pl.col("uid") < 500)
evaluate_model(model, users_history, test_df, k=10)

100%|█████████████████████████████████████████████████████████████████████████████████| 4827/4827 [03:24<00:00, 23.63it/s]

Mean Recall@10: 0.05633607971711287
Mean NDCG@10: 0.10609058390022198


## Content_based 

Content-Based Filtering — это метод рекомендаций, который подбирает объекты, похожие на те, что пользователь уже оценил, используя их содержательные признаки (жанры, embedding-и, особенности контента). Его сильная сторона в том, что он не зависит от других пользователей, хорошо работает для новых объектов (нет cold-start для items) и даёт легко объяснимые рекомендации. Однако у подхода есть ограничения: качество сильно зависит от самих признаков, модель не умеет учитывать тренды или контекст поведения, а рекомендации могут быть слишком однообразными и «сужать» вкусы пользователя.


#### embeddongs 

Методы точного поиска ближайших соседей (kNN) требуют вычислять расстояние между вектором-запросом и каждым объектом в базе, поэтому их сложность линейная 
𝑂
(
𝑁
)
O(N), и на больших наборах данных с десятками или сотнями тысяч элементов они становятся крайне медленными: нужно выполнить миллионы скалярных произведений при каждом запросе. По этой причине в рекомендациях и поиске похожих объектов чаще используют ANN (Approximate Nearest Neighbors) — алгоритмы, которые строят специальный индекс (HNSW, Annoy, Faiss и др.) и ищут ближайших соседей не полностью перебирая весь набор, а умно обходя лишь небольшую часть точек. В результате скорость поиска ускоряется в десятки–сотни раз при минимальной потере качества, что делает ANN практически стандартом для больших эмбеддингов.

In [3]:
def build_users_history_normal(lf: pl.LazyFrame) -> dict:
    """
    Возвращает dict: uid -> set(item_id) только за последние 2 месяца.
    """

    history_df = (
        lf
        .select(["uid", "item_id"])
        .unique()
        .group_by("uid")
        .agg(pl.col("item_id").alias("items"))
        .collect()
    )

    return {
        row["uid"]: set(row["items"])
        for row in history_df.iter_rows(named=True)
    }

In [8]:
class CBF_by_embeding:
    def __init__(self):
        pass

    def fit(self, data, index, item_ids):
        self.data = data

        # self.data = self.data.filter(pl.col("listen_count")>20)

        self.index, self.item_ids = index, item_ids
        self.id2pos = {iid: i for i, iid in enumerate(self.item_ids)} # Индексы реальные и в матрице - разные. Нужно для сопоставления

        
        self.total = []
        self.N = 100
        
        self.history = build_users_history_normal(self.data)
        
    # Профиль пользователя = средний вектор всех айтемов, которые он слушал.
    def build_user_profile_embed(self, uid):

        if uid in self.history:
            
            listened_items = self.history[uid]
        else:
            self.total.append(1)
            return None
    
        indices = [self.id2pos[iid] for iid in listened_items if iid in self.id2pos]
        
        
        if len(indices) <1:
            
            return None
        
        
        vectors = np.vstack([self.index.reconstruct(int(i)) for i in indices]).astype("float32")

        user_vec = np.median(vectors, axis=0)
        user_vec = np.asarray(user_vec)        
                
        return np.array([user_vec])

    def similar_tracks(self, vec): #

        sims, ids = self.index.search(vec, self.N+1)
    
        sims = sims[0]
        ids = ids[0]
        
        result = []
        for track_pos, score in zip(ids, sims):
            result.append((self.item_ids[track_pos], float(score)))

        return [pair[0] for pair in result], [pair[1] for pair in result] 

    def recommend(self, uid):
        vec = self.build_user_profile_embed(uid)
        if vec is None:
            return [], []    
        return self.similar_tracks(vec)

In [6]:
import json
from utils.embeddings import *

with open("data/item_map.json", "r", encoding="utf-8") as f:
    item_map = json.load(f)

item_map = {int(k): v for k, v in item_map.items()}

index, all_item_ids = create_index("data/filtered_embeddings.parquet", item_map)



In [9]:
model = CBF_by_embeding()
model.fit(train_df, index, all_item_ids)

evaluate_model(model, users_history, test_df)

100%|█████████████████████████████████████████████████████████████████████████████████| 4827/4827 [01:34<00:00, 50.87it/s]

Mean Recall@10: 0.0019759325741021926
Mean NDCG@10: 0.004800482900009544


Заметки грустного ML-шика: После первой попытки точность была крайне маленькой

In [93]:
pl.scan_parquet("data/work/train_encoded_lf.parquet").filter(pl.col("item_id") == 218373).collect()  #28556
  # "1114058": 1114058,
  #   "2625039": 1114058,
  #   "4618546": 1114058,

timestamp,is_organic,played_ratio_pct,track_length_seconds,event_type,uid,item_id
u32,u8,u16,u32,enum,i64,i64
7843520,1,5,270,"""listen""",2143,218373


## Ранжирование

Ранжирование (Learning to Rank) — это задача, в которой модель не просто предсказывает значение, а упорядочивает объекты так, чтобы наиболее релевантные оказались выше в списке. Такой подход используется в поиске, рекомендациях и подборе контента — везде, где важна правильная сортировка результатов.

CatBoost хорошо подходит для ранжирования, потому что:

* поддерживает современные алгоритмы LTR (YetiRank, PairLogit и др.);
* отлично работает с категориальными признаками без дополнительной обработки;
* показывает высокую устойчивость и качество «из коробки»;
* легко использовать — достаточно передать группы (group_id).

Поэтому CatBoostRanker — удобная и мощная модель для построения систем рекомендаций и поиска.

### Гибридная модель

Объединием все полученные модели в одну гибридную. Каждая будет выдвать свой топ предсказаний ( но будем брать не 10 а больше). Затем полученные результаты будут уходить на вход модели ранжирования

топ 10 предсказания как правильно все разные - важно отобрать верно нужное

In [22]:
from catboost import CatBoostRanker
from utils.evaluate import filtering_listened_items


class HybridModel:
    def __init__(self):
        self.trend_model = MostPop_by_likes()
        self.new_items_model = NewItemsLastNDays()
        self.als = ALS()

        self.item_knn = ItemKNN()
        self.N = 10
        self.limit = 10
        self.test = False
        self.use_history = False

    
    def fit(self, train_df, train_df_conf, train_df_conf_als):
        print("Prepere history")



        
        history = (
            train_df
            .filter(pl.col("event_type") == "listen")
            # .filter(pl.col("is_organic") == 1)
            .select(["uid", "item_id", "timestamp"])
            .group_by(["uid", "item_id"])
            .agg(pl.col("timestamp").max().alias("last_timestamp"))
            .filter(pl.col("last_timestamp") >(60*60*24*300 - 60*60*24*7))
            .group_by("uid")
            .agg(pl.col("item_id"))      # без .list()
            .collect()
            
        )
        
        # превратить в dict {uid: [item_ids]}
        self.uid_to_items = dict(
            zip(
                history["uid"].to_list(),
                history["item_id"].to_list(),   # это уже списки item_id
            )
        )

        print("Prepere data")
        
        # материализуем, если LazyFrame
        df = train_df_conf.collect() if isinstance(train_df_conf, pl.LazyFrame) else train_df_conf
        
        # dict {(uid, item_id): conf}
        user_item_relevance = {
            (row["uid"], row["item_id"]): row["conf"]
            for row in df.select(["uid", "item_id", "conf"]).iter_rows(named=True)
        }
        
        # список уникальных uid
        hybrid_train_users = df["uid"].unique().to_list()
        
        print("Train recomended models")
        print("trend_model")
        self.trend_model.fit(train_df)
        self.new_items_model.fit(train_df) 

        train_df_conf_als = train_df_conf_als.with_columns(
                (pl.col("conf") != 0).cast(pl.Int8).alias("conf")
            )
        
        self.als.fit(train_df_conf_als)

        
        self.item_knn.fit(train_df)

        
        print("Prepere data for rancin model")
        
        X_rows = []      # сюда складываем фичи (строки)
        y_train = []     # сюда релевантность
        group_id = []    # группы (по пользователю)
        
        current_group = 0

        for user_id in tqdm(hybrid_train_users):
            # кандидаты от гибридной системы
            candidates = self.gather_candidates(user_id)
            
            if not candidates:
                # если модель не смогла ничего отдать – пропускаем пользователя
                continue
        
            rows = self.make_features(user_id, candidates)
            if not rows:
                continue

            n_rows = len(rows)
        
            for row in rows:
                item_id = row["item_id"]
        
                rel = user_item_relevance.get((user_id, item_id), 0.0)
                # print(rel)
                X_rows.append(row)
                y_train.append(rel)
        
            group_id.extend([current_group] * n_rows)
            current_group += 1
        
        self.X_train = pd.DataFrame(X_rows)
        self.y_train = y_train
        self.group_id = group_id
        
        print("Train rancin model")
        
        self.hybrid_model = CatBoostRanker(
            iterations=300,
            learning_rate=0.05,
            depth=6,
            loss_function="YetiRank",
            verbose=50,
        )
        
    def fit_ranker(self):
        self.hybrid_model.fit(
            self.X_train,
            self.y_train,
            group_id=self.group_id,
        )


    def gather_candidates(self, uid):

        candidates = {}
        
        rec, weights = self.trend_model.recommend(uid)
        rec, weights = rec[:self.limit], weights[:self.limit]
    
        for item, score in zip(rec, weights):
            candidates.setdefault(item, {})['trend'] = score
    
        rec, weights = self.new_items_model.recommend(uid)
        rec, weights = rec[:self.limit], weights[:self.limit]
    
        for item, score in zip(rec, weights):
            candidates.setdefault(item, {})['new'] = score

        rec, weights = self.als.recommend(uid)
        rec, weights = rec[:self.limit], weights[:self.limit]
    
        for item, score in zip(rec, weights):
            candidates.setdefault(item, {})['als'] = score

        rec, weights = self.item_knn.recommend(uid)
        rec, weights = rec[:self.limit], weights[:self.limit]
    
        for item, score in zip(rec, weights):
            candidates.setdefault(item, {})['item_knn'] = score
            
            
        return candidates

    def make_features(self, user_id, candidates):
        rows = []
        for item_id, scores in candidates.items():
    
            row = {
                "user_id": user_id,
                "item_id": item_id,
                "score_item_knn":   scores.get("item_knn", 0),
                "score_trend":      scores.get("trend", 0),
                "score_als":   scores.get("als", 0),
                "score_new":   scores.get("new", 0),
            }
    
            rows.append(row)
        
        return rows
            
    def recommend(self, uid):
        
        candidates = self.gather_candidates(uid)
        if not candidates:
           
            return []
        
        rows = self.make_features(uid, candidates)
        if not rows:
            return []
    
        df = pd.DataFrame(rows)
        if self.test:
            print("до", len(df))
        # user_listened_items = get_user_listened_items(self.history, uid)

        if self.use_history:
            if uid in self.uid_to_items:
                df = df[~df["item_id"].isin(self.uid_to_items[uid])]

        if self.test:
             print("после", len(df))
            
        scores = self.hybrid_model.predict(df)
        df["score"] = scores
    
        df_sorted = df.sort_values("score", ascending=False)
        
        return df_sorted["item_id"].tolist(), df_sorted["score"].tolist()  




In [12]:
import numpy as np
import pandas as pd

In [23]:

# ToDo формулу описать!!! зачем логорифирование и описать смысл коэффициентов почему max  
def calculate_conf_hyp(lf: pl.LazyFrame) -> pl.LazyFrame:
    lf = lf.filter(pl.col("played_ratio_max")>50)    
    return lf.with_columns(
        (
            20.0 * pl.col("like_flag").cast(pl.Float64)
            - 10.0 * pl.col("dislike_flag").cast(pl.Float64)
             +(
                pl.col("listen_count").cast(pl.Float64)
                * (pl.col("played_ratio_max").cast(pl.Float64) / 100.0)
            ).log1p()
        ).alias("conf")
    )

train_df_conf = pl.scan_parquet("data/work/train_df_conf.parquet")
train_df = pl.scan_parquet("data/work/train_df_preprocessed.parquet")
train_df_conf_hyp = merge_data_by_count(train_df)
train_df_conf_hyp = calculate_conf_hyp(train_df_conf_hyp)
# train_df_conf_hyp = train_df_conf_hyp.filter((pl.col("listen_count")>5) | (pl.col("like_flag")==1) )


# train_df_conf_hyp.collect()


hybrid = HybridModel()

hybrid.fit(train_df, train_df_conf_hyp, train_df_conf)
 


Prepere history
Prepere data
Train recomended models
trend_model


100%|███████████████████████████████████████████████████████████████████████| 15/15 [00:29<00:00,  1.98s/it, loss=0.00264]


Prepere data for rancin model


100%|█████████████████████████████████████████████████████████████████████████████████| 8896/8896 [10:46<00:00, 13.77it/s]


Train rancin model


In [78]:
hybrid.hybrid_model = CatBoostRanker(
            iterations=3000,
            learning_rate=0.1,
            l2_leaf_reg=0.3,
            depth=10,
            loss_function="YetiRank",
            # verbose=50,
            # logging_level='verbose_eval',
            verbose=True, 
            metric_period=50
    
        )
hybrid.fit_ranker()

0:	total: 148ms	remaining: 7m 24s
50:	total: 7.19s	remaining: 6m 55s
100:	total: 14.4s	remaining: 6m 54s
150:	total: 21.6s	remaining: 6m 48s
200:	total: 29s	remaining: 6m 44s
250:	total: 36.2s	remaining: 6m 36s
300:	total: 43.5s	remaining: 6m 29s
350:	total: 50.6s	remaining: 6m 22s
400:	total: 57.8s	remaining: 6m 14s
450:	total: 1m 4s	remaining: 6m 7s
500:	total: 1m 11s	remaining: 5m 58s
550:	total: 1m 18s	remaining: 5m 50s
600:	total: 1m 26s	remaining: 5m 44s
650:	total: 1m 33s	remaining: 5m 37s
700:	total: 1m 40s	remaining: 5m 30s
750:	total: 1m 47s	remaining: 5m 23s
800:	total: 1m 55s	remaining: 5m 15s
850:	total: 2m 2s	remaining: 5m 8s
900:	total: 2m 9s	remaining: 5m 1s
950:	total: 2m 16s	remaining: 4m 55s
1000:	total: 2m 24s	remaining: 4m 48s
1050:	total: 2m 31s	remaining: 4m 41s
1100:	total: 2m 39s	remaining: 4m 35s
1150:	total: 2m 46s	remaining: 4m 27s
1200:	total: 2m 53s	remaining: 4m 20s
1250:	total: 3m 1s	remaining: 4m 13s
1300:	total: 3m 8s	remaining: 4m 5s
1350:	total: 3m 1

In [80]:
evaluate_model(hybrid, users_history, test_df) #.filter(pl.col("uid") < 1000)

100%|█████████████████████████████████████████████████████████████████████████████████| 4827/4827 [06:58<00:00, 11.55it/s]

Mean Recall@10: 0.041739717449840695
Mean NDCG@10: 0.08076224853666847


## Анализ ошибок 

Точность системы составляла 2.7
Были выбраны пользоваетли для которых есть пересечения из всего списка рекомендаций есть попадания.
Было обнаружено что они находятся на >10 индексах, а между ними в основном треки которые были недавно показаны уже пользователю
Был добавлен фильтр на последние показанные песни 

In [ ]:
# Посмотрим вывод нескольких моделей


In [18]:
from tqdm import tqdm
from utils.metrics import ndcg_at_k, recall_at_k
import numpy as np
import polars as pl

def filtering_listened_items(rec, weights, listened_items: set, k=None):
    filtered_rec = []
    filtered_weights = []

    for item, weight in zip(rec, weights):
        if item in listened_items:
            continue
        filtered_rec.append(item)
        filtered_weights.append(weight)
        if k is not None and len(filtered_rec) >= k:
            break

    return filtered_rec, filtered_weights
    

def evaluate_model(model, user_history, test_lf: pl.LazyFrame, k: int = 10):

    overall_recall = []
    overall_ndcg = []
    test_lf = test_lf.collect()
    for i in tqdm(range(len(test_lf))): 
        
        uid = test_lf[i, "uid"] 
        user_true = set(test_lf[i, "items"] )

        if not user_true:
            continue
        try:
            rec, weights = model.recommend(uid)
        except:
            rec, weights = [], []

        if not rec:
            continue

        recall = recall_at_k(rec, user_true, k)
        ndcg = ndcg_at_k(rec, user_true, k)

        if uid in [6339, 7905, 929]:
            print(uid, ndcg, rec[:10])
        
        overall_recall.append(recall)
        overall_ndcg.append(ndcg)

    print("Mean Recall@{}:".format(k), np.mean(overall_recall))
    print("Mean NDCG@{}:".format(k), np.mean(overall_ndcg))

In [39]:
results = {
    "MostPop": {
        929: {
            "score": 0.06943122193677727,
            "items": [679713, 368953, 492083, 587807, 77971, 584672, 661967, 272391, 673338, 221077]
        },
        7905: {
            "score": 0.0,
            "items": [679713, 368953, 492083, 587807, 77971, 584672, 661967, 272391, 673338, 221077]
        },
        6339: {
            "score": 0.3589542101716347,
            "items": [679713, 368953, 492083, 587807, 77971, 584672, 661967, 272391, 673338, 221077]
        },
    },

    "MostPop_by_listen": {
        929: {
            "score": 0.0,
            "items": [368953, 665904, 867033, 587807, 221077, 627154, 810966, 241997, 511362, 618038]
        },
        7905: {
            "score": 0.0,
            "items": [368953, 665904, 867033, 587807, 221077, 627154, 810966, 241997, 511362, 618038]
        },
        6339: {
            "score": 0.22009176629808017,
            "items": [368953, 665904, 867033, 587807, 221077, 627154, 810966, 241997, 511362, 618038]
        },
    },

    "NewItemsLastNDays": {
        929: {
            "score": 0.0,
            "items": [679713, 77971, 492083, 833867, 574324, 305144, 85378, 95745, 17549, 758397]
        },
        7905: {
            "score": 0.0,
            "items": [679713, 77971, 492083, 833867, 574324, 305144, 85378, 95745, 17549, 758397]
        },
        6339: {
            "score": 0.22009176629808017,
            "items": [679713, 77971, 492083, 833867, 574324, 305144, 85378, 95745, 17549, 758397]
        },
    },

    "LastListenRecommender": {
        929: {
            "score": 0.22009176629808017,
            "items": [351253, 589562, 801191, 546554, 837025, 476671, 665904, 587807, 55570, 386337]
        },
        7905: {
            "score": 0.36211082347570717,
            "items": [689616, 449874, 473339, 173724, 30334, 760035, 524855, 223455, 892412, 86749]
        },
        6339: {
            "score": 0.43735247915031,
            "items": [824785, 341666, 74061, 272391, 234840, 838111, 425069, 585096, 98030, 126064]
        },
    },

    "ALS": {
        929: {
            "score": 0.0,
            "items": [348693, 266016, 609187, 43405, 445383, 910003, 143673, 705928, 146600, 615306]
        },
        7905: {
            "score": 0.0,
            "items": [609187, 627154, 146600, 736229, 43405, 131632, 526016, 574331, 810966, 601953]
        },
        6339: {
            "score": 0.06362078819895171,
            "items": [402248, 842667, 333315, 568302, 269741, 791362, 25993, 626744, 587763, 723026]
        },
    },

    "ItemKNN": {
        929: {
            "score": 0.0,
            "items": [609187, 43405, 574331, 589562, 266016, 627154, 389120, 131632, 720251, 801191]
        },
        7905: {
            "score": 0.0,
            "items": [273689, 712340, 117709, 389120, 195103, 283212, 91936, 889665, 698779, 911161]
        },
        6339: {
            "score": 0.22009176629808017,
            "items": [824785, 266016, 627154, 609187, 131632, 389120, 665904, 231245, 43405, 514880]
        },
    },

    "RandomWalkWithRestart": {
        929: {
            "score": 0.0,
            "items": [4292, 386337, 445383, 864368, 929389, 280741, 817116, 439202, 436510, 644682]
        },
        7905: {
            "score": 0.08514311764162098,
            "items": [574730, 449874, 502682, 152442, 689616, 30334, 887245, 896175, 304528, 12550]
        },
        6339: {
            "score": 0.23297260855707355,
            "items": [665904, 812032, 873196, 306377, 128290, 61617, 703194, 833248, 861821, 7654]
        },
    },
}



real = {
    929: [
        49480, 117709, 351253, 349176, 272391, 443975, 831161, 751383, 772200,
        724402, 648159, 644713, 534092
    ],

    7905: [
        6196, 67596, 97545, 112009, 86749, 225566, 355578, 371114, 371114,
        371114, 371114, 235307, 264794, 435380, 418343, 444097, 452208,
        419096, 445679, 820576, 797965, 852535, 584166, 582358, 584790,
        569017, 601934, 616374, 710903, 732562, 760035, 764852, 667252,
        634537, 659866, 689616, 628074, 628074, 628074, 628074, 628074,
        919502, 886213, 896685, 479918, 517109, 371114, 584166
    ],

    6339: [
        5674, 31505, 61617, 67302, 128290, 124896, 133701, 341666, 368953,
        327773, 385078, 312725, 321277, 426277, 778003, 824785, 838111,
        838111, 833248, 833248, 583461, 592555, 593426, 577188, 613770,
        704037, 727362, 762540, 715675, 723026, 752174, 645684, 650772,
        653245, 625883, 624295, 625325, 679713, 683185, 659705, 918977,
        907237, 470336, 524462, 503402
    ],
}
    

Идеальный мир, в котором умеем отбирать кандиндатов и имеем идеальную ранжирующую модель.
При этом есть пересечения LastListenRecommender и других моделей. В преспективе модель поможет отсеивать уже просмотренные и более релевантые выйдут в топ

In [79]:
uid  = 6339
methods = [
    "MostPop",
    "NewItemsLastNDays",
    "LastListenRecommender",
    "ALS",
    "ItemKNN",
    "RandomWalkWithRestart",
]

all_pred = [item
            for m in methods
            for item in results[m][uid]["items"]]

all_pred = list(set(all_pred) & set(real[uid]))
ndcg_at_k(all_pred, set(real[uid]), k = 10)
# len(set(all_pred) & set(real[uid])) / len(set(real[uid]))
# all_pred


0.9363792118010483

In [32]:
# model = RandomWalkWithRestart()
# model.fit(lf_user_track)

# # test_users_items_df.filter(pl.col("uid") < 500)
# evaluate_model(model, users_history, test_df, k=10)

In [ ]:
real = pl.scan_parquet("data/work/test_df_preprocessed_for_eval.parquet").filter(pl.col("uid") == 729).select("items").collect().to_series().to_list()[0]  #28556
# pl.scan_parquet("data/work/test_df_preprocessed.parquet").filter(pl.col("uid") == 729).filter(pl.col("item_id") == 855098).collect()  #28556
# not_like = lf_user_track.filter(pl.col("played_ratio_max") < 50).filter(pl.col("uid") == 123).select(pl.col("item_id").unique()).collect().to_series().to_list()

# pl.scan_parquet("data/work/test_df_preprocessed.parquet").filter(pl.col("is_organic") == 0).collect()

In [31]:
print(pl.scan_parquet("data/work/test_df_preprocessed_for_eval.parquet").filter(pl.col("uid") == 6339).select("items").collect().to_series().to_list()[0])

[5674, 31505, 61617, 67302, 128290, 124896, 133701, 341666, 368953, 327773, 385078, 312725, 321277, 426277, 778003, 824785, 838111, 838111, 833248, 833248, 583461, 592555, 593426, 577188, 613770, 704037, 727362, 762540, 715675, 723026, 752174, 645684, 650772, 653245, 625883, 624295, 625325, 679713, 683185, 659705, 918977, 907237, 470336, 524462, 503402]
